In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "Proj1_test1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler -- Randomly sampling C and Max_Iter values
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,300)
    }
)

# Specify a Policy -- Bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env
                      )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = experiment.submit(hyperdrive_run_config)

# Monitor HyperDrive runs 
# You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

#Best Run and metrics
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

#save the model
best_run.register_model(model_name = "hyperdrive_best_run.pkl", model_path = './outputs/')

print(best_run)
best_run.download_file( name= './outputs/hyper_model_AzureML.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###